In [ ]:
#Import necessary modules and packages
%matplotlib qt5
import numpy as np
import scipy.signal
from matplotlib import pyplot as plt

# Defining functions

In [ ]:
# BUTTER HIGHPASSFILTER
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(order, normal_cutoff, btype = "high", analog = False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y

# Load data and create time series

In [ ]:
Displacements = np.loadtxt(open("filename", "rb"), delimiter=",")
points = np.loadtxt(open("filename", "rb"), delimiter=",")

# Sampling frequency/fps
Fs = 220

dt = 1/Fs
T = dt*len(Displacements[0])
t = np.linspace(0, T, len(Displacements[0]))

# Plot displacements 

In [ ]:
#Plot Displacement 
if len(Displacements) == 1:
    plt.figure()
    plt.plot(t,Displacements[0])
    plt.grid()
    plt.xlabel('Time [s]')
    plt.ylabel('Response [pixels]')
    plt.title('Sensor 1')

else:
    # Plot in time domain
    fig, axs = plt.subplots(len(Displacements))
    plt.subplots_adjust(hspace=1)

    for i in range(len(Displacements)):
        axs[i].plot(t,Displacements[i])
        axs[i].set_title(f"sensor %d" % i)
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Response')
        axs[i].grid()
plt.show()

# Estimate and plot spectrums using Welch' method and calculate coherence

In [ ]:
# Apply butter highpass filter
filtered_signal = butter_highpass_filter(Displacements, 1.5, Fs)

# Estimate the spectrum using Welch' method

# Smoothing of spectrum
Nwelch=4 # Number of divisions of time series for averaging of spectrum
Nwindow=np.floor(np.max(np.shape(Displacements[0]))/Nwelch); # Length of window

#Empty variables
Spectrum = {}
peaks = {}
Value = {}
count = 0
diag = []
for n in range(len(Displacements)):   
    # Estimate spectrums
    for h in range(len(Displacements)):
        f, Cross = scipy.signal.csd(filtered_signal[n], filtered_signal[h],Fs,nperseg=Nwindow)
        Spectrum[count] = Cross 
        if n == h:
            diag.append(count)   
        count = count + 1   
        
# Coherence function
Coher = []
count_coher = 0
# Find coherence 
for i in range(len(diag)):
    for h in range(len(diag)):
        coher = np.abs(Spectrum[count_coher])**2/(Spectrum[diag[i]]*Spectrum[diag[h]])
        Coher.append(coher)
        count_coher = count_coher + 1

# Plot spectrums and coherence plot

In [ ]:
#Plot PSD in frequency domain    
if len(Displacements) == 1:
    plt.figure()
    plt.plot(f,np.real(Spectrum[0]))
    plt.grid()
    plt.xlabel('f [Hz]')
    plt.ylabel('PSD')
    plt.title('Sensor 1')
    #plt.yscale("log")

else:
    fig, axs = plt.subplots(len(Displacements),len(Displacements))
    plt.subplots_adjust(hspace=1)
    count = 0
    for b in range(len(Displacements)):
        for k in range(len(Displacements)):
            axs[b,k].plot(f,np.real(Spectrum[count]))
            count=count+1
            axs[b,k].set_title(f"sensor %d %d" %(b,k))
            axs[b,k].grid()
plt.show()

# Plot coherence     
fig, axs = plt.subplots(len(diag),len(diag))
plt.subplots_adjust(hspace=1)
count_coher=-1
for b in range(len(diag)):
    for k in range(len(diag)):
        count_coher = count_coher + 1
        if k !=b:
            axs[b,k].plot(f,np.real(Coher[count_coher]))
            axs[b,k].set_title(f"sensor %d %d" %(b,k))
            axs[b,k].grid()  
        else:
            axs[b,k].plot(f,np.real(Spectrum[diag[b]]))
            axs[b,k].set_title(f"sensor %d %d" %(b,k))
            axs[b,k].grid()       
plt.show()

# Peak picking

Use the computer mouse to pick the peaks that represent the natural frequencies. The peaks can be picked from every plot.

In [ ]:
fig, axs = plt.subplots(len(diag),1)
plt.subplots_adjust(hspace=1)
count = 0
for i in range(len(diag)):
    axs[i].semilogy(f,np.real(Spectrum[diag[count]]))
    count = count + 1
    axs[i].grid()
selected = plt.ginput(n=-1, timeout = 0)

# Calculate and plot mode shapes

In [ ]:
val = input("Is the displacements in x- or y-direction?: ")
Scale = float(input("How much would you like to amplify the Mode shapes?: "))

diff = []
x_orig = []
y_orig = []
for i in range(len(points)):
    if val =='x':
        x_orig.append(points[i][0])
        y_orig.append(-points[i][1])
        diff.append(max(Displacements[i])-min(Displacements[i]))
        max_value = max(diff)
        max_index = diff.index(max_value)
    else: 
        x_orig.append(points[i][0])
        y_orig.append(points[i][1])
        diff.append(max(Displacements[i])-min(Displacements[i]))
        max_value = max(diff)
        max_index = diff.index(max_value)

#Find frequencies from chosen peaks
Frequencies = []     
for i in range(len(selected)):
    Frequencies.append(selected[i][0])

#Find index of nearest value
index = []
for k in range(len(Frequencies)):
    difference_array = np.absolute(f-Frequencies[k])
    index.append(difference_array.argmin())
Variable = index

#Find mode shapes
ModeShapes = np.zeros([len(Frequencies),len(points)]) # Create empty mode shapes

for k in range(len(Frequencies)):
    count = max_index
    for m in range(len(points)-1):
        ModeShapes[k][m] = np.sqrt(np.real(Spectrum[diag[m]][Variable[k]])/np.real(Spectrum[diag[max_index]][Variable[k]]))*np.sign(np.real(Spectrum[count][Variable[k]]))
        count = count+len(points)  

#Scale mode shapes
ModeShapes = ModeShapes*Scale

#Create x_values for plotting undeformed shape
x_point = round(np.mean(x_orig))
x_coordinates_orig = []
for i in range(len(x_orig)):
    x_coordinates_orig.append(x_point)

#Plot mode shapes
for n in range(len(Frequencies)):
    x_points=[]
    y_points=[]
    plt.figure()
    plt.grid()
    plt.title("Modeshape frequency. %f Hz." %Frequencies[n] + "  Scaled by %i"  %Scale )
    for p in range(len(points)):
        if val == 'x':
            x_points.append(ModeShapes[n][p]+points[p][0])
            y_points.append(-points[p][1])
        else:
            y_points.append(ModeShapes[n][p]+points[p][1])
            x_points.append(points[p][0])
            
    x_points[len(x_points)-1] = x_coordinates_orig[0]
    plt.plot(x_points, y_points, 'o', linestyle="-", color='orange', label='Mode shape')
    plt.plot(x_coordinates_orig, y_orig, 'o', linestyle="-", color='black', label='Original Shape')
    plt.legend()

# Short time Fourier transform

In [ ]:
# Short time Fourier transfomrs
Signal_stft = butter_highpass_filter(Displacements, 1.5, Fs)

amp = 0.005
f, t, Zxx = scipy.signal.stft(Signal_stft[0], Fs, nperseg=1000)

figsize = np.array([1, 1/1.618]) * 11/2.5
fig, ax = plt.subplots(figsize=figsize, dpi=300)
ax.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=amp, shading='gouraud')

plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.tight_layout()